In [11]:
import time
import numpy as np
import pandas as pd
from scipy import signal
import tensorflow as tf
import os
import sys
import json

from sklearn.metrics import accuracy_score
from sklearn.preprocessing import label_binarize
from sklearn.cross_validation import StratifiedKFold, KFold, train_test_split

import keras.backend as K
from keras.models import Sequential, Model, model_from_json
from keras.layers.core import Dense, Activation, Dropout
# from keras.layers.core import Dense, Activation, Merge, Dropout
from keras.layers import LSTM, Input, Lambda, Concatenate, Conv1D, AveragePooling1D, MaxPooling1D
from keras.layers.wrappers import TimeDistributed
from keras.layers.normalization import BatchNormalization
from keras.optimizers import SGD, Adam, RMSprop

sys.path.append("./")
from utilities.utils import *

import matplotlib
import matplotlib.pyplot as plt
import matplotlib.ticker as ticker
%matplotlib inline

from IPython.display import clear_output

# pip install soundfile
import soundfile

In [12]:
import sys
print(sys.version)
print('Tensorflow:',tf.__version__)

3.6.1 |Anaconda custom (64-bit)| (default, May 11 2017, 13:09:58) 
[GCC 4.4.7 20120313 (Red Hat 4.4.7-1)]
Tensorflow: 1.5.0-dev20171102


In [13]:
# def pad_sequence_into_array(Xs, maxlen=None, truncating='post', padding='post', value=0.):
#     """
#     Padding sequence (list of numpy arrays) into an numpy array
#     :param Xs: list of numpy arrays. The arrays must have the same shape except the first dimension.
#     :param maxlen: the allowed maximum of the first dimension of Xs's arrays. Any array longer than maxlen is truncated to maxlen
#     :param truncating: = 'pre'/'post', indicating whether the truncation happens at either the beginning or the end of the array (default)
#     :param padding: = 'pre'/'post',indicating whether the padding happens at either the beginning or the end of the array (default)
#     :param value: scalar, the padding value, default = 0.0
#     :return: Xout, the padded sequence (now an augmented array with shape (Narrays, N1stdim, N2nddim, ...)
#     :return: mask, the corresponding mask, binary array, with shape (Narray, N1stdim)
#     """
#     Nsamples = len(Xs)
#     if maxlen is None:
#         lengths = [s.shape[0] for s in Xs]    # 'sequences' must be list, 's' must be numpy array, len(s) return the first dimension of s
#         maxlen = np.max(lengths)

#     Xout = np.ones(shape=[Nsamples, maxlen] + list(Xs[0].shape[1:]), dtype=Xs[0].dtype) * np.asarray(value, dtype=Xs[0].dtype)
#     Mask = np.zeros(shape=[Nsamples, maxlen], dtype=Xout.dtype)
#     for i in range(Nsamples):
#         x = Xs[i]
#         if truncating == 'pre':
#             trunc = x[-maxlen:]
#         elif truncating == 'post':
#             trunc = x[:maxlen]
#         else:
#             raise ValueError("Truncating type '%s' not understood" % truncating)
#         if padding == 'post':
#             Xout[i, :len(trunc)] = trunc
#             Mask[i, :len(trunc)] = 1
#         elif padding == 'pre':
#             Xout[i, -len(trunc):] = trunc
#             Mask[i, -len(trunc):] = 1
#         else:
#             raise ValueError("Padding type '%s' not understood" % padding)
#     return Xout, Mask

In [14]:
# y = [np.array([1]),np.array([0]),np.array([1])]
# y, y_mask = pad_sequence_into_array(y, maxlen=1)

In [15]:
# print(y)

In [16]:
# print(y_mask)

# Model definition

In [17]:
def ctc_lambda_func(args):
    y_pred, labels, input_length, label_length = args
    # the 2 is critical here since the first couple outputs of the RNN
    # tend to be garbage:
    shift = 2
    y_pred = y_pred[:, shift:, :]
    input_length -= shift
    print("ctc_lambda_func checking input_length[0]: ", input_length[0])
    print("ctc_lambda_func checking y_pred.shape: ", y_pred.shape)
    return K.ctc_batch_cost(labels, y_pred, input_length, label_length)

In [18]:
def build_model(nb_feat, nb_class, optimizer='Adadelta'):
#     training = Input(name='training_phase', shape=[1], dtype='bool')
#     if training == False:
#         K.set_training_phase(0)
        
    net_input = Input(name="the_input", shape=(32000, nb_feat))
    
    avg_pool_1 = AveragePooling1D(pool_size=2, padding='same'
                        )(net_input)
    
    temp_conv_1 = Conv1D(filters = 32, 
                         kernel_size = [40],
                         padding='same',
                         activation='relu', 
                         use_bias=True
                        )(avg_pool_1)
    
    temp_conv_1 = BatchNormalization()(temp_conv_1)
    
    dropout_1 = Dropout(0.0)(temp_conv_1)
    
    max_pool_1 = MaxPooling1D(pool_size=2, padding='same'
                        )(dropout_1)
    
        
    temp_conv_2 = Conv1D(filters = 32, 
                         kernel_size = [40],
                         padding='same',
                         activation='relu', 
                         use_bias=True
                        )(max_pool_1)
    
    temp_conv_2 = BatchNormalization()(temp_conv_2)
    
    dropout_2 = Dropout(0.1)(temp_conv_2)
    
    max_pool_2 = MaxPooling1D(pool_size=2, padding='same'
                        )(dropout_2)
       
        
    temp_conv_3 = Conv1D(filters = 32, 
                         kernel_size = [40],
                         padding='same',
                         activation='relu', 
                         use_bias=True
                        )(max_pool_2)
    
    temp_conv_3 = BatchNormalization()(temp_conv_3)
    
    dropout_3 = Dropout(0.1)(temp_conv_3)
    
    max_pool_3 = MaxPooling1D(pool_size=2, padding='same'
                        )(dropout_3)
    
        
    temp_conv_4 = Conv1D(filters = 32, 
                         kernel_size = [40],
                         padding='same',
                         activation='relu', 
                         use_bias=True
                        )(max_pool_3)
    
    temp_conv_4 = BatchNormalization()(temp_conv_4)
    
    dropout_4 = Dropout(0.2)(temp_conv_4)
    
    max_pool_4 = MaxPooling1D(pool_size=2, padding='same'
                        )(dropout_4)

    temp_conv_5 = Conv1D(filters = 32, 
                     kernel_size = [40],
                     padding='same',
                     activation='relu', 
                     use_bias=True
                    )(max_pool_4)
    
    temp_conv_5 = BatchNormalization()(temp_conv_5)
    
    dropout_5 = Dropout(0.2)(temp_conv_5)
    
    max_pool_5 = MaxPooling1D(pool_size=2, padding='same'
                        )(dropout_5)
    
        
    temp_conv_6 = Conv1D(filters = 32, 
                         kernel_size = [40],
                         padding='same',
                         activation='relu', 
                         use_bias=True
                        )(max_pool_5)
    
    temp_conv_6 = BatchNormalization()(temp_conv_6)
    
    dropout_6 = Dropout(0.3)(temp_conv_6)
    
    max_pool_6 = MaxPooling1D(pool_size=2, padding='same'
                        )(dropout_6)
    
  
    
        
    temp_conv_7 = Conv1D(filters = 32, 
                         kernel_size = [40],
                         padding='same',
                         activation='relu', 
                         use_bias=True
                        )(max_pool_6)
    
    temp_conv_7 = BatchNormalization()(temp_conv_7)
    
    dropout_7 = Dropout(0.3)(temp_conv_7)
    
    max_pool_7 = MaxPooling1D(pool_size=2, padding='same'
                        )(dropout_7)
    
        
    temp_conv_8 = Conv1D(filters = 32, 
                         kernel_size = [40],
                         padding='same',
                         activation='relu', 
                         use_bias=True
                        )(max_pool_7)
    
    temp_conv_8 = BatchNormalization()(temp_conv_8)
    
    dropout_8 = Dropout(0.4)(temp_conv_8)
    
    max_pool_8 = MaxPooling1D(pool_size=2, padding='same'
                        )(dropout_8)
    
    dropout_9 = Dropout(0.5)(max_pool_8)
    
#     conv layers to do implicit feature extraction from raw wave input
    
    
#     from CTC RNN paper
#     net_input = Input(name="the_input", shape=(78, nb_feat))
    forward_lstm1  = LSTM(output_dim=64, 
                          return_sequences=True, 
                          activation="tanh"
                         )(dropout_9)
#     backward_lstm1 = LSTM(output_dim=64, 
#                           return_sequences=True, 
#                           activation="tanh",
#                           go_backwards=True
#                          )(net_input)
#     blstm_output1  = Concatenate()([forward_lstm1, backward_lstm1])
#     blstm_output1  = Merge(mode='concat')([forward_lstm1, backward_lstm1])
    

    forward_lstm2  = LSTM(output_dim=64, 
                          return_sequences=True, 
                          activation="tanh"
                         )(forward_lstm1)
#     backward_lstm2 = LSTM(output_dim=64, 
#                           return_sequences=True, 
#                           activation="tanh",
#                           go_backwards=True
#                          )(blstm_output1)
#     blstm_output2  = Concatenate()([forward_lstm2, backward_lstm2])
#     blstm_output2  = Merge(mode='concat')([forward_lstm2, backward_lstm2])

    hidden = TimeDistributed(Dense(512, activation='tanh'))(forward_lstm2)
    output = TimeDistributed(Dense(nb_class + 1, activation='softmax'))(hidden)

    labels = Input(name='the_labels', shape=[1], dtype='float32')
    input_length = Input(name='input_length', shape=[1], dtype='int64')
    label_length = Input(name='label_length', shape=[1], dtype='int64')
    loss_out = Lambda(ctc_lambda_func, output_shape=(1,), name="ctc")([output, labels, input_length, label_length])

    model = Model(input=[net_input, labels, input_length, label_length], output=[loss_out, output])
    model.compile(loss={'ctc': lambda y_true, y_pred: y_pred}, optimizer=optimizer, metrics=[])
#     y_pred == loss_out in this case

    test_func = K.function([net_input], [output])
    
    return model, test_func

In [19]:
# raw sound amplitude
nb_feat = 1

nb_class = 2

optimizer = 'Adadelta'

In [20]:
model, test_func = build_model(nb_feat=nb_feat, nb_class=nb_class, optimizer=optimizer)
model.summary()

/home/cheeseprata/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:140: UserWarning: Update your `LSTM` call to the Keras 2 API: `LSTM(return_sequences=True, activation="tanh", units=64)`
/home/cheeseprata/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:153: UserWarning: Update your `LSTM` call to the Keras 2 API: `LSTM(return_sequences=True, activation="tanh", units=64)`


ctc_lambda_func checking input_length[0]:  Tensor("ctc_1/strided_slice_1:0", shape=(1,), dtype=int64)
ctc_lambda_func checking y_pred.shape:  (?, 61, 3)
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
the_input (InputLayer)          (None, 32000, 1)     0                                            
__________________________________________________________________________________________________
average_pooling1d_2 (AveragePoo (None, 16000, 1)     0           the_input[0][0]                  
__________________________________________________________________________________________________
conv1d_9 (Conv1D)               (None, 16000, 32)    1312        average_pooling1d_2[0][0]        
__________________________________________________________________________________________________
batch_normalization_9 (BatchNor (None, 16000, 32)    12

/home/cheeseprata/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:171: UserWarning: Update your `Model` call to the Keras 2 API: `Model(inputs=[<tf.Tenso..., outputs=[<tf.Tenso...)`
/home/cheeseprata/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:172: UserWarning: Output "time_distributed_4" missing from loss dictionary. We assume this was done on purpose, and we will not be expecting any data to be passed to "time_distributed_4" during training.


# Data preparation

In [21]:
import pickle

def create_dataset_from_wavs(folder_name, save_as='.pkl', seed=13):

    X = []
    
    for audio_file in sorted(os.listdir( os.path.join('../Datasets', folder_name) )):
        filename_stub, ext = os.path.splitext(audio_file)
        if not (ext=='.wav' or ext=='.ogg'): continue
            
        samples, sample_rate = soundfile.read( os.path.join('../Datasets', folder_name, audio_file) )

        #         slow but accurate way
        #         samples_16khz = resample_sample_data(samples, sample_rate, 16000)
        
        target_sample_rate_multiplier = sample_rate // 16000
        
#         start resampling at 1s mark
        start_sample_ind = (sample_rate * 1) -1
    
        end_sample_ind = start_sample_ind + (sample_rate * 2)
        
        samples_16khz = samples[start_sample_ind:end_sample_ind:target_sample_rate_multiplier]
        
        if np.amax(samples_16khz) > -(np.amin(samples_16khz)):
            samples_16khz = samples_16khz / np.asscalar(np.amax(samples_16khz))
        else:
            samples_16khz = samples_16khz / np.asscalar(-(np.amin(samples_16khz)))
            
        if samples_16khz.shape == (32000,):
#             samples_16khz.reshape(32000,1)
            print("resampling to samples_16khz", filename_stub, sample_rate, len(samples), samples.shape, len(samples_16khz), samples_16khz.shape)
            X.append(samples_16khz)

    ds_file = os.path.join('../Datasets', folder_name+"raw_sample"+save_as)
    pickle.dump(X, open(ds_file, 'wb'), protocol=pickle.HIGHEST_PROTOCOL)
    print("Created dataset : %s" % (ds_file, ))
#     X = np.asarray(X)
    return X

In [22]:
def resample_sample_data(samples, initial_sample_rate, target_sample_rate):
    num_target_sample = len(samples) * target_sample_rate // initial_sample_rate
    return signal.resample(samples, num_target_sample)

In [23]:
# # from CTC paper's utils.py
# def get_sample(ids, take_all=False, params=Constants()):
#     if take_all:
#         files = os.listdir(params.path_to_features)
#         ids = np.sort([f[:-4] for f in files if f.endswith(".csv")])
#     tx = []
#     ty = []
#     valid_ids = []
#     for i in ids:
#         x, y = load_sample(params.path_to_features + i + '.csv')
#         if len(x) > 0:
#             tx.append(np.array(x, dtype=float))
#             ty.append(y[0])
#             valid_ids.append(i)
#     tx = np.array(tx)
#     ty = np.array(ty)
#     return tx, ty, np.array(valid_ids)


In [24]:
# # X, y, valid_idxs = get_sample(ids=None, take_all=True)
# y = np.argmax(to_categorical(y, params), axis=1)
# y = np.reshape(y, (y.shape[0], 1))

In [25]:
# X, X_mask = pad_sequence_into_array(X, maxlen=78)
# y, y_mask = pad_sequence_into_array(y, maxlen=1)

In [26]:
# index_to_retain = np.sum(X_mask, axis=1, dtype=np.int32) > 5

In [27]:
# X, X_mask = X[index_to_retain], X_mask[index_to_retain]
# y, y_mask = y[index_to_retain], y_mask[index_to_retain]

In [28]:
# idxs_train, idxs_test = train_test_split(range(X.shape[0]))
# X_train, X_test = X[idxs_train], X[idxs_test]
# X_train_mask, X_test_mask = X_mask[idxs_train], X_mask[idxs_test]
# y_train, y_test = y[idxs_train], y[idxs_test]
# y_train_mask, y_test_mask = y_mask[idxs_train], y_mask[idxs_test]

In [29]:
# Created
first_folder_name = 'happy'
X_happy = create_dataset_from_wavs(first_folder_name)
Y_happy = np.ones(shape=[len(X_happy)], dtype=np.int) * np.asarray(0, dtype=np.int)


resampling to samples_16khz 03-01-03-01-01-01-01 48000 166567 (166567,) 32000 (32000,)
resampling to samples_16khz 03-01-03-01-01-01-02 48000 172973 (172973,) 32000 (32000,)
resampling to samples_16khz 03-01-03-01-01-01-03 48000 180980 (180980,) 32000 (32000,)
resampling to samples_16khz 03-01-03-01-01-01-04 48000 168168 (168168,) 32000 (32000,)
resampling to samples_16khz 03-01-03-01-01-01-05 48000 192192 (192192,) 32000 (32000,)
resampling to samples_16khz 03-01-03-01-01-01-06 48000 169769 (169769,) 32000 (32000,)
resampling to samples_16khz 03-01-03-01-01-01-07 48000 168168 (168168,) 32000 (32000,)
resampling to samples_16khz 03-01-03-01-01-01-08 48000 168168 (168168,) 32000 (32000,)
resampling to samples_16khz 03-01-03-01-01-01-09 48000 160160 (160160,) 32000 (32000,)
resampling to samples_16khz 03-01-03-01-01-01-10 48000 168168 (168168,) 32000 (32000,)
resampling to samples_16khz 03-01-03-01-01-01-11 48000 148949 (148949,) 32000 (32000,)
resampling to samples_16khz 03-01-03-01-01-

resampling to samples_16khz 03-01-03-02-01-01-15 48000 171371 (171371,) 32000 (32000,)
resampling to samples_16khz 03-01-03-02-01-01-16 48000 180981 (180981,) 32000 (32000,)
resampling to samples_16khz 03-01-03-02-01-01-17 48000 179379 (179379,) 32000 (32000,)
resampling to samples_16khz 03-01-03-02-01-01-18 48000 180981 (180981,) 32000 (32000,)
resampling to samples_16khz 03-01-03-02-01-01-19 48000 180981 (180981,) 32000 (32000,)
resampling to samples_16khz 03-01-03-02-01-01-20 48000 176176 (176176,) 32000 (32000,)
resampling to samples_16khz 03-01-03-02-01-01-21 48000 182582 (182582,) 32000 (32000,)
resampling to samples_16khz 03-01-03-02-01-01-22 48000 179380 (179380,) 32000 (32000,)
resampling to samples_16khz 03-01-03-02-01-01-23 48000 155356 (155356,) 32000 (32000,)
resampling to samples_16khz 03-01-03-02-01-01-24 48000 192192 (192192,) 32000 (32000,)
resampling to samples_16khz 03-01-03-02-01-02-01 48000 174574 (174574,) 32000 (32000,)
resampling to samples_16khz 03-01-03-02-01-

In [30]:
X_happy

[array([ 0.02295253,  0.02243088, -0.00938967, ...,  0.        ,
         0.        ,  0.        ]),
 array([ 0.        , -0.00018208, -0.00018208, ...,  0.        ,
         0.        ,  0.00018208]),
 array([ 0.        ,  0.        , -0.00030506, ...,  0.00183038,
         0.00030506,  0.00152532]),
 array([ 0.        , -0.00037064,  0.        , ...,  0.        ,
        -0.00037064,  0.        ]),
 array([-0.00322407, -0.00268673, -0.00214938, ..., -0.0177324 ,
        -0.02471789, -0.02901666]),
 array([ 0.        ,  0.00047348,  0.        , ...,  0.00236742,
         0.        ,  0.        ]),
 array([ 0.00239425,  0.00558659,  0.00478851, ...,  0.        ,
         0.        ,  0.        ]),
 array([-0.00388538, -0.00582807, -0.00291404, ..., -0.00242836,
        -0.00242836, -0.00291404]),
 array([ 0.00126263,  0.00126263,  0.00252525, ...,  0.00883838,
         0.01010101,  0.01262626]),
 array([ 0.00011184,  0.        ,  0.00011184, ...,  0.        ,
         0.00011184,  0.00

In [31]:
X_happy = np.array(X_happy)

In [32]:
len(X_happy)

191

In [33]:
X_happy.shape

(191, 32000)

In [34]:
X_happy[0]

array([ 0.02295253,  0.02243088, -0.00938967, ...,  0.        ,
        0.        ,  0.        ])

In [35]:
Y_happy.shape

(191,)

In [36]:
Y_happy[0]

0

In [37]:
second_folder_namename = 'angry'
X_angry = create_dataset_from_wavs(second_folder_namename)
Y_angry = np.ones(shape=[len(X_angry)], dtype=np.int) * np.asarray(1, dtype=np.int)

resampling to samples_16khz 03-01-05-01-01-01-01 48000 185786 (185786,) 32000 (32000,)
resampling to samples_16khz 03-01-05-01-01-01-02 48000 176176 (176176,) 32000 (32000,)
resampling to samples_16khz 03-01-05-01-01-01-03 48000 192192 (192192,) 32000 (32000,)
resampling to samples_16khz 03-01-05-01-01-01-04 48000 177778 (177778,) 32000 (32000,)
resampling to samples_16khz 03-01-05-01-01-01-05 48000 185786 (185786,) 32000 (32000,)
resampling to samples_16khz 03-01-05-01-01-01-06 48000 180981 (180981,) 32000 (32000,)
resampling to samples_16khz 03-01-05-01-01-01-07 48000 192192 (192192,) 32000 (32000,)
resampling to samples_16khz 03-01-05-01-01-01-08 48000 177778 (177778,) 32000 (32000,)
resampling to samples_16khz 03-01-05-01-01-01-09 48000 171371 (171371,) 32000 (32000,)
resampling to samples_16khz 03-01-05-01-01-01-10 48000 187387 (187387,) 32000 (32000,)
resampling to samples_16khz 03-01-05-01-01-01-11 48000 172973 (172973,) 32000 (32000,)
resampling to samples_16khz 03-01-05-01-01-

resampling to samples_16khz 03-01-05-02-01-01-21 48000 209810 (209810,) 32000 (32000,)
resampling to samples_16khz 03-01-05-02-01-01-22 48000 185786 (185786,) 32000 (32000,)
resampling to samples_16khz 03-01-05-02-01-01-23 48000 163363 (163363,) 32000 (32000,)
resampling to samples_16khz 03-01-05-02-01-01-24 48000 209810 (209810,) 32000 (32000,)
resampling to samples_16khz 03-01-05-02-01-02-01 48000 209810 (209810,) 32000 (32000,)
resampling to samples_16khz 03-01-05-02-01-02-02 48000 192192 (192192,) 32000 (32000,)
resampling to samples_16khz 03-01-05-02-01-02-03 48000 211412 (211412,) 32000 (32000,)
resampling to samples_16khz 03-01-05-02-01-02-04 48000 185785 (185785,) 32000 (32000,)
resampling to samples_16khz 03-01-05-02-01-02-05 48000 171371 (171371,) 32000 (32000,)
resampling to samples_16khz 03-01-05-02-01-02-06 48000 201801 (201801,) 32000 (32000,)
resampling to samples_16khz 03-01-05-02-01-02-07 48000 192192 (192192,) 32000 (32000,)
resampling to samples_16khz 03-01-05-02-01-

In [38]:
len(X_angry)

192

In [39]:
Y_angry.shape

(192,)

In [40]:
Y_angry[0]

1

In [41]:
X_angry = np.array(X_angry)

In [42]:
X_happy

array([[ 0.02295253,  0.02243088, -0.00938967, ...,  0.        ,
         0.        ,  0.        ],
       [ 0.        , -0.00018208, -0.00018208, ...,  0.        ,
         0.        ,  0.00018208],
       [ 0.        ,  0.        , -0.00030506, ...,  0.00183038,
         0.00030506,  0.00152532],
       ..., 
       [ 0.        , -0.00018864,  0.        , ...,  0.00547067,
         0.00660253,  0.00660253],
       [ 0.00024257,  0.        ,  0.        , ...,  0.        ,
         0.        ,  0.        ],
       [ 0.00011605,  0.00011605,  0.        , ..., -0.0004642 ,
        -0.0004642 , -0.00011605]])

In [43]:
X_angry

array([[  0.00000000e+00,  -1.73852573e-04,   0.00000000e+00, ...,
          0.00000000e+00,   0.00000000e+00,   0.00000000e+00],
       [  0.00000000e+00,   0.00000000e+00,  -1.99760288e-04, ...,
          0.00000000e+00,   0.00000000e+00,   0.00000000e+00],
       [ -5.65006075e-02,   6.07533414e-02,   1.45808019e-02, ...,
         -1.70109356e-02,  -2.36938032e-02,  -2.30862697e-02],
       ..., 
       [  0.00000000e+00,   0.00000000e+00,   0.00000000e+00, ...,
          7.14709176e-02,   6.97582504e-02,  -1.25156446e-03],
       [  0.00000000e+00,  -9.39849624e-05,   0.00000000e+00, ...,
          0.00000000e+00,   0.00000000e+00,   0.00000000e+00],
       [ -2.07447360e-04,  -1.03723680e-04,  -2.07447360e-04, ...,
          8.29789441e-04,   1.14096048e-03,   1.65957888e-03]])

In [44]:
X = np.concatenate((X_happy, X_angry))

In [45]:
X

array([[  2.29525300e-02,   2.24308816e-02,  -9.38967136e-03, ...,
          0.00000000e+00,   0.00000000e+00,   0.00000000e+00],
       [  0.00000000e+00,  -1.82083030e-04,  -1.82083030e-04, ...,
          0.00000000e+00,   0.00000000e+00,   1.82083030e-04],
       [  0.00000000e+00,   0.00000000e+00,  -3.05064063e-04, ...,
          1.83038438e-03,   3.05064063e-04,   1.52532032e-03],
       ..., 
       [  0.00000000e+00,   0.00000000e+00,   0.00000000e+00, ...,
          7.14709176e-02,   6.97582504e-02,  -1.25156446e-03],
       [  0.00000000e+00,  -9.39849624e-05,   0.00000000e+00, ...,
          0.00000000e+00,   0.00000000e+00,   0.00000000e+00],
       [ -2.07447360e-04,  -1.03723680e-04,  -2.07447360e-04, ...,
          8.29789441e-04,   1.14096048e-03,   1.65957888e-03]])

In [46]:
len(X)

383

In [47]:
X = np.asarray(X)

In [48]:
X.shape

(383, 32000)

In [49]:
y = np.append(Y_happy, Y_angry)

In [50]:
y.shape

(383,)

In [51]:
y

array([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1,

In [52]:
y = y.reshape((-1, 1))

In [53]:
y

array([[0],
       [0],
       [0],
       [0],
       [0],
       [0],
       [0],
       [0],
       [0],
       [0],
       [0],
       [0],
       [0],
       [0],
       [0],
       [0],
       [0],
       [0],
       [0],
       [0],
       [0],
       [0],
       [0],
       [0],
       [0],
       [0],
       [0],
       [0],
       [0],
       [0],
       [0],
       [0],
       [0],
       [0],
       [0],
       [0],
       [0],
       [0],
       [0],
       [0],
       [0],
       [0],
       [0],
       [0],
       [0],
       [0],
       [0],
       [0],
       [0],
       [0],
       [0],
       [0],
       [0],
       [0],
       [0],
       [0],
       [0],
       [0],
       [0],
       [0],
       [0],
       [0],
       [0],
       [0],
       [0],
       [0],
       [0],
       [0],
       [0],
       [0],
       [0],
       [0],
       [0],
       [0],
       [0],
       [0],
       [0],
       [0],
       [0],
       [0],
       [0],
       [0],
       [0],
    

In [54]:
y.shape

(383, 1)

In [55]:
X = X.reshape((X.shape[0], X.shape[1], 1))

In [56]:
idxs_train, idxs_test = train_test_split(range(X.shape[0]))
X_train, X_test = X[idxs_train], X[idxs_test]
# X_train_mask, X_test_mask = X_mask[idxs_train], X_mask[idxs_test]
y_train, y_test = y[idxs_train], y[idxs_test]
# y_train_mask, y_test_mask = y_mask[idxs_train], y_mask[idxs_test]

In [57]:
X_train.shape

(287, 32000, 1)

In [58]:
X_train[0]

array([[-0.00083099],
       [-0.01642802],
       [-0.02454615],
       ..., 
       [ 0.01380721],
       [ 0.0146382 ],
       [ 0.00952442]])

In [59]:
X_train[0][10000]

array([-0.12471235])

In [60]:
X_train[0:10].shape

(10, 32000, 1)

In [61]:
X_test.shape

(96, 32000, 1)

In [62]:
y_train.shape

(287, 1)

In [63]:
y_test.shape

(96, 1)

In [64]:
y_test

array([[0],
       [0],
       [1],
       [1],
       [1],
       [0],
       [1],
       [0],
       [0],
       [1],
       [0],
       [0],
       [0],
       [0],
       [0],
       [0],
       [1],
       [0],
       [0],
       [1],
       [0],
       [1],
       [0],
       [1],
       [0],
       [1],
       [1],
       [0],
       [1],
       [0],
       [0],
       [1],
       [0],
       [1],
       [1],
       [0],
       [0],
       [1],
       [1],
       [1],
       [1],
       [0],
       [0],
       [0],
       [1],
       [1],
       [0],
       [1],
       [1],
       [0],
       [0],
       [0],
       [1],
       [1],
       [1],
       [1],
       [1],
       [0],
       [0],
       [0],
       [0],
       [1],
       [1],
       [1],
       [1],
       [1],
       [1],
       [0],
       [0],
       [0],
       [0],
       [1],
       [0],
       [1],
       [0],
       [0],
       [0],
       [1],
       [0],
       [1],
       [1],
       [0],
       [1],
    

In [65]:
# X_train_input_lengths = np.asarray([( ((len(x) -79) // 2) - 3999 // 40) for x in X_train])
X_train_input_lengths = np.asarray([63 for x in X_train])

In [66]:
X_train_input_lengths.shape

(287,)

In [67]:
X_train_input_lengths[0]

63

In [68]:
# X_test_input_lengths = np.asarray([len(x) for x in X_test])
X_test_input_lengths = np.asarray([63 for x in X_test])

In [69]:
X_test_input_lengths.shape

(96,)

In [70]:
X_test_input_lengths[0]

63

In [71]:
X_train_input_lengths

array([63, 63, 63, 63, 63, 63, 63, 63, 63, 63, 63, 63, 63, 63, 63, 63, 63,
       63, 63, 63, 63, 63, 63, 63, 63, 63, 63, 63, 63, 63, 63, 63, 63, 63,
       63, 63, 63, 63, 63, 63, 63, 63, 63, 63, 63, 63, 63, 63, 63, 63, 63,
       63, 63, 63, 63, 63, 63, 63, 63, 63, 63, 63, 63, 63, 63, 63, 63, 63,
       63, 63, 63, 63, 63, 63, 63, 63, 63, 63, 63, 63, 63, 63, 63, 63, 63,
       63, 63, 63, 63, 63, 63, 63, 63, 63, 63, 63, 63, 63, 63, 63, 63, 63,
       63, 63, 63, 63, 63, 63, 63, 63, 63, 63, 63, 63, 63, 63, 63, 63, 63,
       63, 63, 63, 63, 63, 63, 63, 63, 63, 63, 63, 63, 63, 63, 63, 63, 63,
       63, 63, 63, 63, 63, 63, 63, 63, 63, 63, 63, 63, 63, 63, 63, 63, 63,
       63, 63, 63, 63, 63, 63, 63, 63, 63, 63, 63, 63, 63, 63, 63, 63, 63,
       63, 63, 63, 63, 63, 63, 63, 63, 63, 63, 63, 63, 63, 63, 63, 63, 63,
       63, 63, 63, 63, 63, 63, 63, 63, 63, 63, 63, 63, 63, 63, 63, 63, 63,
       63, 63, 63, 63, 63, 63, 63, 63, 63, 63, 63, 63, 63, 63, 63, 63, 63,
       63, 63, 63, 63, 63

In [72]:
X_test_input_lengths

array([63, 63, 63, 63, 63, 63, 63, 63, 63, 63, 63, 63, 63, 63, 63, 63, 63,
       63, 63, 63, 63, 63, 63, 63, 63, 63, 63, 63, 63, 63, 63, 63, 63, 63,
       63, 63, 63, 63, 63, 63, 63, 63, 63, 63, 63, 63, 63, 63, 63, 63, 63,
       63, 63, 63, 63, 63, 63, 63, 63, 63, 63, 63, 63, 63, 63, 63, 63, 63,
       63, 63, 63, 63, 63, 63, 63, 63, 63, 63, 63, 63, 63, 63, 63, 63, 63,
       63, 63, 63, 63, 63, 63, 63, 63, 63, 63, 63])

In [73]:
X_test_input_lengths -2

array([61, 61, 61, 61, 61, 61, 61, 61, 61, 61, 61, 61, 61, 61, 61, 61, 61,
       61, 61, 61, 61, 61, 61, 61, 61, 61, 61, 61, 61, 61, 61, 61, 61, 61,
       61, 61, 61, 61, 61, 61, 61, 61, 61, 61, 61, 61, 61, 61, 61, 61, 61,
       61, 61, 61, 61, 61, 61, 61, 61, 61, 61, 61, 61, 61, 61, 61, 61, 61,
       61, 61, 61, 61, 61, 61, 61, 61, 61, 61, 61, 61, 61, 61, 61, 61, 61,
       61, 61, 61, 61, 61, 61, 61, 61, 61, 61, 61])

# Training

In [74]:
batch_size = 8
nb_epoch = 20

In [75]:
sess = tf.Session()

In [76]:
# class_weights = np.unique(y, return_counts=True)[1]*1.
# class_weights = np.sum(class_weights) / class_weights

# sample_weight = np.zeros(y_train.shape[0])
# for num, i in enumerate(y_train):
#     sample_weight[num] = class_weights[i[0]]

In [77]:
ua_train = np.zeros(nb_epoch)
ua_test = np.zeros(nb_epoch)
wa_train = np.zeros(nb_epoch)
wa_test = np.zeros(nb_epoch)
loss_train = np.zeros(nb_epoch)
loss_test = np.zeros(nb_epoch)

for epoch in range(nb_epoch):
    print("starting epoch #", epoch)
    epoch_time0 = time.time()
    
    total_ctcloss = 0.0
    batches = range(0, X_train.shape[0], batch_size)
    shuffle = np.random.choice(batches, size=len(batches), replace=False)
    print("shuffle: ", shuffle)
    print("shuffle.shape: ", shuffle.shape)
#     K.set_learning_phase(1)
    for num, i in enumerate(shuffle):
        print("X_train[i:i+batch_size]: ", X_train[i:i+batch_size])
        print("y_train[i:i+batch_size] ", y_train[i:i+batch_size])
        inputs_train = {'the_input': X_train[i:i+batch_size],
                        'the_labels': y_train[i:i+batch_size],
                        'input_length': X_train_input_lengths[i:i+batch_size],
                        'label_length': np.ones((y_train[i:i+batch_size].shape[0]), dtype=np.int),
                       }
        outputs_train = {'ctc': np.zeros([inputs_train["the_labels"].shape[0]])}
#         K.set_learning_phase(1)
        ctcloss, _ = model.train_on_batch(x=inputs_train, y=outputs_train, 
#                                        sample_weight=sample_weight[i:i+batch_size]
                                      )

        total_ctcloss += ctcloss * inputs_train["the_input"].shape[0] * 1.
        print("X_train[i:i+batch_size]: ", X_train[i:i+batch_size])
        print("y_train[i:i+batch_size]: ", y_train[i:i+batch_size])
        print("ctcloss: ", ctcloss)
        print("batch #", num)
    print("batch training complete for epoch #", epoch)
#     K.set_learning_phase(0)
    loss_train[epoch] = total_ctcloss / X_train.shape[0]

    print("print point #1")
    inputs_train = {'the_input': X_train[:10],
                    'the_labels': y_train[:10],
                    'input_length': X_train_input_lengths[:10],
                    'label_length': np.ones((y_train[:10].shape[0]), dtype=np.int),
                   }
    outputs_train = {'ctc': np.zeros([y_train.shape[0]])}
    print("print point #2")
#     preds = test_func([inputs_train["the_input"]])[0]
    ctcloss, preds = model.predict(inputs_train)
    print("train ctcloss : ", ctcloss)
    print("train preds.shape : ", preds.shape)
#     preds = preds_outputs[0]
#     preds = test_func([inputs_train["the_input"], False])[0]
    print("train preds :", preds)
    print("np.argmax(preds, 2) : ", np.argmax(preds, 2))
    print("preds.shape :", preds.shape)
    print("print point #3")
    decode_function = K.ctc_decode(preds[:,2:,:], inputs_train["input_length"]-2, greedy=False, top_paths=1)
#     decode_function = K.ctc_decode(preds[:,2:,:], inputs_train["input_length"]-2)
    print("print point #4")
    labellings = decode_function[0][0].eval(session=sess)
    print("train inputs_train[the_labels]: ", inputs_train["the_labels"])
    print("train labellings: ", labellings)
    print("train labellings.shape: ", labellings.shape)
#     print("train labellings.T[0]: ", labellings.T[0])
#     print("train labellings.T[0].ravel(): ", labellings.T[0].ravel())
    print("y_train.ravel(): ", y_train.ravel())
    
    if labellings.shape[1] == 1:
        print("train labellings.shape[1] == 1")
        print("test y_test.ravel():", y_train[:10].ravel())
        print("test labellings.ravel():", labellings.ravel())
        ua_train[epoch] = accuracy_score(y_train[:10].ravel(), labellings.ravel())
        wa_train[epoch] = accuracy_score(y_train[:10].ravel(), labellings.ravel())
    else:
        ua_train[epoch] = 0.0
        wa_train[epoch] = 0.0
#     if labellings.shape[1] == 0:
#         ua_train[epoch] = 0.0
#         wa_train[epoch] = 0.0
#     else:
#         ua_train[epoch] = unweighted_accuracy(y_train.ravel(), labellings.T[0].ravel())
#         wa_train[epoch] = weighted_accuracy(y_train.ravel(), labellings.T[0].ravel())

#     ua_train[epoch] = 0.0
#     wa_train[epoch] = 0.0


    print("print point #5")
    inputs_test = {'the_input': X_test,
                   'the_labels': y_test,
                   'input_length': X_test_input_lengths,
                   'label_length': np.ones((X_test.shape[0]), dtype=np.int),
                  }
    outputs_test = {'ctc': np.zeros([y_test.shape[0]])}
#     preds = test_func([inputs_test["the_input"]])[0]
    ctcloss, preds = model.predict(inputs_test)
    print("test ctcloss : ", ctcloss)
    print("test preds.shape : ", preds.shape)
    print("test preds : ", preds)
    print("np.argmax(preds, 2) : ", np.argmax(preds, 2))
#     preds = preds_outputs[0]
#     preds = test_func([inputs_test["the_input"], False])[0]
    decode_function = K.ctc_decode(preds[:,2:,:], inputs_test["input_length"]-2, greedy=False, top_paths=1)
#     decode_function = K.ctc_decode(preds[:,2:,:], inputs_test["input_length"]-2)
    labellings = decode_function[0][0].eval(session=sess)
    print("test inputs_test[the_labels]: ", inputs_test["the_labels"])
    print("test labellings: ", labellings)
    print("test labellings.shape: ", labellings.shape)
#     print("test labellings.T[0]: ", labellings.T[0])
#     print("test labellings.T[0].ravel(): ", labellings.T[0].ravel())
    print("y_test.ravel(): ", y_test.ravel())
    
#     if labellings.shape[1] == 0:
#         ua_test[epoch] = 0.0
#         wa_test[epoch] = 0.0
#     else:
#         ua_test[epoch] = unweighted_accuracy(y_test.ravel(), labellings.T[0].ravel())
#         wa_test[epoch] = weighted_accuracy(y_test.ravel(), labellings.T[0].ravel())
    if labellings.shape[1] == 1:
        print("test labellings.shape[1] == 1")
        print("test y_test.ravel():", y_test.ravel())
        print("test labellings.ravel():", labellings.ravel())
        ua_test[epoch] = accuracy_score(y_test.ravel(), labellings.ravel())
        wa_test[epoch] = accuracy_score(y_test.ravel(), labellings.ravel())
    else:
        ua_test[epoch] = 0.0
        wa_test[epoch] = 0.0
        
#     ua_test[epoch] = 0.0
#     wa_test[epoch] = 0.0
    test_ctc_loss, _ = model.predict(inputs_test)
    loss_test[epoch] = np.mean(test_ctc_loss)

    epoch_time1 = time.time()


    print("ending epoch #", epoch)
    print('epoch = %d, \
WA Tr = %0.2f, UA Tr = %0.2f, WA Te = %0.2f, UA Te = %0.2f, CTC Tr = %0.2f, CTC Te = %0.2f, \
time = %0.2fmins' % (epoch + 1, 
                     wa_train[epoch], ua_train[epoch], 
                     wa_test[epoch], ua_test[epoch], 
                     loss_train[epoch], loss_test[epoch],
                     (epoch_time1-epoch_time0)/60))

starting epoch # 0
shuffle:  [ 72  16  96  64 264 224 176 160  80 136  40   8 192 104 184 272 248 280
 152  48 256 120  56 200 168 144  24 208 216 240 128 112   0 232  88  32]
shuffle.shape:  (36,)
X_train[i:i+batch_size]:  [[[  7.36366794e-02]
  [ -1.41368531e-01]
  [  8.16255644e-03]
  ..., 
  [  8.68357068e-04]
  [  1.04202848e-03]
  [  1.04202848e-03]]

 [[  1.93928709e-02]
  [ -1.98887114e-02]
  [  3.28907498e-02]
  ..., 
  [  5.50933833e-05]
  [  5.50933833e-05]
  [  5.50933833e-05]]

 [[ -3.13479624e-04]
  [ -9.40438871e-04]
  [  0.00000000e+00]
  ..., 
  [ -1.88087774e-03]
  [ -2.19435737e-03]
  [  4.70219436e-03]]

 ..., 
 [[  0.00000000e+00]
  [  0.00000000e+00]
  [  0.00000000e+00]
  ..., 
  [  0.00000000e+00]
  [  0.00000000e+00]
  [  0.00000000e+00]]

 [[  2.39425379e-03]
  [  5.58659218e-03]
  [  4.78850758e-03]
  ..., 
  [  0.00000000e+00]
  [  0.00000000e+00]
  [  0.00000000e+00]]

 [[  1.12346927e-04]
  [  0.00000000e+00]
  [  1.12346927e-04]
  ..., 
  [ -3.37040782e-0

X_train[i:i+batch_size]:  [[[  0.00000000e+00]
  [  0.00000000e+00]
  [  0.00000000e+00]
  ..., 
  [  1.57927985e-04]
  [  0.00000000e+00]
  [  0.00000000e+00]]

 [[  0.00000000e+00]
  [  0.00000000e+00]
  [  0.00000000e+00]
  ..., 
  [  0.00000000e+00]
  [  3.99201597e-04]
  [  3.99201597e-04]]

 [[ -4.36331903e-04]
  [  1.41080649e-02]
  [  1.73805541e-02]
  ..., 
  [ -9.67202385e-03]
  [ -2.13075413e-02]
  [ -1.66169733e-02]]

 ..., 
 [[ -1.92975685e-04]
  [ -1.92975685e-04]
  [  0.00000000e+00]
  ..., 
  [  0.00000000e+00]
  [  1.92975685e-04]
  [  1.92975685e-04]]

 [[  0.00000000e+00]
  [  9.66245813e-05]
  [  0.00000000e+00]
  ..., 
  [ -3.86498325e-04]
  [  1.28832775e-04]
  [  3.22081938e-05]]

 [[ -2.10128178e-04]
  [  0.00000000e+00]
  [  2.10128178e-04]
  ..., 
  [  0.00000000e+00]
  [  0.00000000e+00]
  [  0.00000000e+00]]]
y_train[i:i+batch_size]:  [[0]
 [0]
 [1]
 [1]
 [1]
 [0]
 [1]
 [1]]
ctcloss:  27.5906
batch # 5
X_train[i:i+batch_size]:  [[[  0.00000000e+00]
  [  2.92

X_train[i:i+batch_size]:  [[[ -1.69252468e-02]
  [  1.38692995e-02]
  [  1.17536436e-02]
  ..., 
  [  0.00000000e+00]
  [ -9.40291490e-04]
  [  9.40291490e-04]]

 [[  1.19918455e-03]
  [  1.79877683e-03]
  [  1.43902147e-03]
  ..., 
  [  8.39429188e-04]
  [  6.59551505e-04]
  [  4.79673822e-04]]

 [[  0.00000000e+00]
  [  0.00000000e+00]
  [  0.00000000e+00]
  ..., 
  [  1.07503763e-04]
  [  2.15007525e-04]
  [  0.00000000e+00]]

 ..., 
 [[ -7.02621820e-03]
  [ -7.28644851e-03]
  [ -6.96116063e-03]
  ..., 
  [  0.00000000e+00]
  [  0.00000000e+00]
  [  6.50575760e-05]]

 [[  4.04639871e-04]
  [ -1.34879957e-04]
  [ -1.34879957e-04]
  ..., 
  [  0.00000000e+00]
  [ -4.04639871e-04]
  [ -9.44159698e-04]]

 [[ -2.86115007e-02]
  [  4.11407200e-03]
  [  5.92800374e-02]
  ..., 
  [ -4.20757363e-03]
  [ -4.30107527e-03]
  [ -4.48807854e-03]]]
y_train[i:i+batch_size]:  [[0]
 [0]
 [0]
 [0]
 [1]
 [1]
 [0]
 [0]]
ctcloss:  2.32036
batch # 10
X_train[i:i+batch_size]:  [[[ -2.34387805e-02]
  [  6.0

X_train[i:i+batch_size]:  [[[-0.00012012]
  [ 0.        ]
  [ 0.00024024]
  ..., 
  [ 0.        ]
  [ 0.        ]
  [ 0.        ]]

 [[-0.02364865]
  [-0.09346847]
  [-0.08671171]
  ..., 
  [-0.00225225]
  [-0.00112613]
  [-0.00112613]]

 [[-0.00074405]
  [-0.00074405]
  [-0.00044643]
  ..., 
  [-0.00595238]
  [-0.00580357]
  [-0.00595238]]

 ..., 
 [[ 0.00061013]
  [ 0.        ]
  [ 0.        ]
  ..., 
  [ 0.        ]
  [ 0.        ]
  [ 0.        ]]

 [[ 0.00186829]
  [ 0.00093414]
  [ 0.00280243]
  ..., 
  [-0.02802429]
  [-0.02755722]
  [-0.02101822]]

 [[-0.09103896]
  [-0.07532468]
  [-0.06571429]
  ..., 
  [ 0.00038961]
  [ 0.00038961]
  [ 0.00038961]]]
y_train[i:i+batch_size]:  [[0]
 [1]
 [0]
 [0]
 [0]
 [0]
 [0]
 [0]]
ctcloss:  1.71802
batch # 15
X_train[i:i+batch_size]:  [[[ -2.17202433e-04]
  [  0.00000000e+00]
  [  0.00000000e+00]
  ..., 
  [  3.25803649e-04]
  [  5.43006082e-04]
  [  7.60208514e-04]]

 [[  1.54297176e-04]
  [  0.00000000e+00]
  [  3.08594353e-04]
  ..., 
  

X_train[i:i+batch_size]:  [[[ -1.20036934e-02]
  [ -9.23361034e-03]
  [ -1.01569714e-02]
  ..., 
  [ -9.23361034e-04]
  [ -9.23361034e-04]
  [  0.00000000e+00]]

 [[  6.33512829e-05]
  [  2.21729490e-04]
  [  2.21729490e-04]
  ..., 
  [  1.49762433e-01]
  [  4.76401647e-02]
  [ -7.23471650e-02]]

 [[ -1.03369201e-01]
  [ -5.75057329e-02]
  [  3.72199682e-02]
  ..., 
  [  1.58228965e-01]
  [  3.15399541e-01]
  [  3.19456694e-01]]

 ..., 
 [[  2.49816944e-03]
  [  2.28281001e-03]
  [  2.15359435e-03]
  ..., 
  [  5.85777663e-03]
  [  6.37463927e-03]
  [  6.58999871e-03]]

 [[ -7.30140187e-05]
  [ -2.19042056e-04]
  [  0.00000000e+00]
  ..., 
  [  7.30140187e-05]
  [  2.19042056e-04]
  [  7.30140187e-05]]

 [[  2.41165469e-02]
  [  1.01387932e-02]
  [ -1.84073236e-02]
  ..., 
  [  0.00000000e+00]
  [  1.96869771e-04]
  [  0.00000000e+00]]]
y_train[i:i+batch_size]:  [[0]
 [1]
 [0]
 [0]
 [0]
 [1]
 [1]
 [0]]
ctcloss:  1.5974
batch # 20
X_train[i:i+batch_size]:  [[[-0.00059952]
  [-0.00209832

...

X_train[i:i+batch_size]:  [[[ -8.30989517e-04]
  [ -1.64280235e-02]
  [ -2.45461519e-02]
  ..., 
  [  1.38072104e-02]
  [  1.46381999e-02]
  [  9.52441831e-03]]

 [[ -9.59752322e-02]
  [ -1.25386997e-01]
  [ -3.27657379e-01]
  ..., 
  [  5.15995872e-04]
  [  0.00000000e+00]
  [  0.00000000e+00]]

 [[  0.00000000e+00]
  [  0.00000000e+00]
  [ -3.67782273e-04]
  ..., 
  [  0.00000000e+00]
  [  0.00000000e+00]
  [  0.00000000e+00]]

 ..., 
 [[  0.00000000e+00]
  [ -1.99322304e-04]
  [ -3.98644608e-04]
  ..., 
  [  0.00000000e+00]
  [  0.00000000e+00]
  [  0.00000000e+00]]

 [[ -2.72294078e-03]
  [ -6.80735194e-04]
  [ -6.80735194e-04]
  ..., 
  [ -2.04220558e-03]
  [ -2.04220558e-03]
  [ -2.04220558e-03]]

 [[  1.80766450e-04]
  [  9.03832249e-05]
  [  9.03832249e-05]
  ..., 
  [  5.42299349e-04]
  [  3.61532899e-04]
  [  1.80766450e-04]]]
y_train[i:i+batch_size]:  [[1]
 [0]
 [0]
 [1]
 [1]
 [1]
 [0]
 [1]]
ctcloss:  0.322981
batch # 16
X_train[i:i+batch_size]:  [[[  7.54242615e-03]
  [  1.

X_train[i:i+batch_size]:  [[[-0.00837851]
  [-0.02464268]
  [ 0.01084278]
  ..., 
  [ 0.00246427]
  [ 0.00246427]
  [ 0.00344998]]

 [[ 0.00895409]
  [ 0.0091446 ]
  [ 0.00857306]
  ..., 
  [ 0.00285769]
  [ 0.00171461]
  [-0.00076205]]

 [[-0.00057883]
  [-0.00057883]
  [-0.00054668]
  ..., 
  [-0.00061099]
  [-0.00048236]
  [-0.00035373]]

 ..., 
 [[ 0.        ]
  [-0.00024802]
  [-0.00024802]
  ..., 
  [-0.00099206]
  [ 0.        ]
  [ 0.        ]]

 [[ 0.        ]
  [ 0.00017603]
  [ 0.00035205]
  ..., 
  [ 0.0007041 ]
  [ 0.00052808]
  [ 0.00035205]]

 [[ 0.        ]
  [ 0.        ]
  [ 0.        ]
  ..., 
  [ 0.        ]
  [ 0.00015763]
  [ 0.00015763]]]
y_train[i:i+batch_size]:  [[1]
 [0]
 [1]
 [1]
 [0]
 [0]
 [0]
 [0]]
ctcloss:  0.532664
batch # 21
X_train[i:i+batch_size]:  [[[  8.73362445e-03]
  [  1.03215562e-02]
  [  5.55776102e-03]
  ..., 
  [ -1.19094879e-03]
  [ -9.52759031e-03]
  [ -4.76379516e-03]]

 [[  0.00000000e+00]
  [  0.00000000e+00]
  [ -9.59692898e-04]
  ..., 
 

X_train[i:i+batch_size]:  [[[ -1.71959188e-04]
  [ -2.86598647e-04]
  [ -5.73197295e-05]
  ..., 
  [  4.81485727e-03]
  [  4.64289809e-03]
  [  0.00000000e+00]]

 [[ -1.85942730e-03]
  [ -2.08255857e-03]
  [ -2.60319821e-03]
  ..., 
  [  0.00000000e+00]
  [  0.00000000e+00]
  [  7.43770919e-05]]

 [[  8.57351967e-03]
  [ -1.60317848e-03]
  [ -1.01069947e-03]
  ..., 
  [  0.00000000e+00]
  [  0.00000000e+00]
  [  0.00000000e+00]]

 ..., 
 [[ -1.40331182e-03]
  [ -7.01655908e-04]
  [  0.00000000e+00]
  ..., 
  [  0.00000000e+00]
  [  0.00000000e+00]
  [  0.00000000e+00]]

 [[ -1.86135198e-03]
  [ -1.77055432e-03]
  [ -1.81595315e-03]
  ..., 
  [  4.81227584e-03]
  [  4.53988287e-03]
  [  3.67730513e-03]]

 [[  1.61479149e-04]
  [  2.01848936e-04]
  [  1.61479149e-04]
  ..., 
  [ -2.53804852e-01]
  [ -3.72411287e-01]
  [ -4.87344072e-01]]]
y_train[i:i+batch_size]:  [[1]
 [1]
 [1]
 [0]
 [1]
 [0]
 [1]
 [1]]
ctcloss:  0.0228052
batch # 26
X_train[i:i+batch_size]:  [[[ -1.55872496e-04]
  [  0

X_train[i:i+batch_size]:  [[[-0.00029727]
  [-0.0008918 ]
  [-0.00059453]
  ..., 
  [ 0.        ]
  [ 0.        ]
  [ 0.        ]]

 [[-0.00125054]
  [-0.00149103]
  [ 0.00033668]
  ..., 
  [-0.0020201 ]
  [-0.0020682 ]
  [-0.0020201 ]]

 [[ 0.00094922]
  [ 0.00142383]
  [ 0.        ]
  ..., 
  [ 0.        ]
  [ 0.        ]
  [ 0.        ]]

 ..., 
 [[ 0.09736308]
  [ 0.06828938]
  [ 0.07167005]
  ..., 
  [ 0.00135227]
  [ 0.00135227]
  [ 0.00067613]]

 [[ 0.        ]
  [ 0.        ]
  [ 0.        ]
  ..., 
  [ 0.07147092]
  [ 0.06975825]
  [-0.00125156]]

 [[ 0.        ]
  [-0.00015124]
  [-0.00015124]
  ..., 
  [ 0.        ]
  [ 0.00287356]
  [ 0.00423472]]]
y_train[i:i+batch_size]:  [[0]
 [1]
 [0]
 [1]
 [1]
 [0]
 [1]
 [1]]
ctcloss:  0.241101
batch # 31
X_train[i:i+batch_size]:  [[[ -1.20036934e-02]
  [ -9.23361034e-03]
  [ -1.01569714e-02]
  ..., 
  [ -9.23361034e-04]
  [ -9.23361034e-04]
  [  0.00000000e+00]]

 [[  6.33512829e-05]
  [  2.21729490e-04]
  [  2.21729490e-04]
  ..., 
 

train inputs_train[the_labels]:  [[1]
 [0]
 [0]
 [1]
 [1]
 [1]
 [0]
 [1]
 [1]
 [0]]
train labellings:  [[ 1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1]
 [ 0 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1]
 [ 1  0  1  0  1  0  1  0 -1 -1 -1 -1 -1 -1 -1 -1]
 [ 1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1]
 [ 1  0  1  0 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1]
 [ 1  0  1  0 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1]
 [ 0 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1]
 [ 1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1]
 [ 1  0  1  0  1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1]
 [ 1  0  1  0  1  0  1  0  1  0  1  0  1  0  1  0]]
train labellings.shape:  (10, 16)
y_train.ravel():  [1 0 0 1 1 1 0 1 1 0 1 0 0 0 0 0 1 1 1 1 0 1 1 1 1 1 1 0 1 0 0 1 1 1 1 1 0
 1 0 1 0 0 0 0 1 1 0 0 1 1 0 1 1 1 0 1 0 0 1 1 0 0 0 0 1 1 0 0 0 0 0 1 1 1
 1 1 0 1 0 0 0 1 0 1 1 1 0 1 1 0 1 0 0 0 1 1 0 0 0 1 0 0 1 0 1 0 1 1 1 0 1
 1 0 1 0 1 1 0 1 1 1 1 0 1 1 1 1 1 0 0 1 0 1 1 0 0 0 1 0 0 1 1 0 0 0 0 1 0
 0 1 0 1 1 0 0 0 0 1 0 0 1 0 1 1 0

In [78]:
# decode_function = K.ctc_decode(preds[:,2:,:], inputs_test["input_length"]-2, greedy=False, top_paths=1)
decode_function = K.ctc_decode(preds[:,2:,:], inputs_test["input_length"]-2)
labellings = decode_function[0][0].eval(session=sess)

In [79]:
print("labellings: ", labellings)

labellings:  [[ 1  1  1  0 -1 -1 -1]
 [ 0 -1 -1 -1 -1 -1 -1]
 [ 1 -1 -1 -1 -1 -1 -1]
 [ 1  0 -1 -1 -1 -1 -1]
 [ 1 -1 -1 -1 -1 -1 -1]
 [ 1 -1 -1 -1 -1 -1 -1]
 [ 1 -1 -1 -1 -1 -1 -1]
 [ 0 -1 -1 -1 -1 -1 -1]
 [ 0  1  0 -1 -1 -1 -1]
 [ 1  1 -1 -1 -1 -1 -1]
 [ 1  1  1  0 -1 -1 -1]
 [ 1  0  1  0 -1 -1 -1]
 [ 1  1 -1 -1 -1 -1 -1]
 [ 0  1  0 -1 -1 -1 -1]
 [ 0 -1 -1 -1 -1 -1 -1]
 [ 0  1  0 -1 -1 -1 -1]
 [ 0  1  1 -1 -1 -1 -1]
 [ 0 -1 -1 -1 -1 -1 -1]
 [ 1  0 -1 -1 -1 -1 -1]
 [ 1 -1 -1 -1 -1 -1 -1]
 [ 1  1  0  1  0 -1 -1]
 [ 1 -1 -1 -1 -1 -1 -1]
 [ 1  1  1 -1 -1 -1 -1]
 [ 1 -1 -1 -1 -1 -1 -1]
 [ 1 -1 -1 -1 -1 -1 -1]
 [ 1 -1 -1 -1 -1 -1 -1]
 [ 1  1  1 -1 -1 -1 -1]
 [ 0  0 -1 -1 -1 -1 -1]
 [ 1  1  1  0 -1 -1 -1]
 [ 0  0  1 -1 -1 -1 -1]
 [ 1 -1 -1 -1 -1 -1 -1]
 [ 1  1 -1 -1 -1 -1 -1]
 [ 0  0 -1 -1 -1 -1 -1]
 [ 1  1 -1 -1 -1 -1 -1]
 [ 1  1  0 -1 -1 -1 -1]
 [ 0 -1 -1 -1 -1 -1 -1]
 [ 0 -1 -1 -1 -1 -1 -1]
 [ 1  1  1  1  1 -1 -1]
 [ 1  1 -1 -1 -1 -1 -1]
 [ 1 -1 -1 -1 -1 -1 -1]
 [ 1  0  1  0 -1 -1 -1]
 [ 

In [80]:
labellings.shape

(96, 7)

In [81]:
preds[:,2:,:]

array([[[  2.04760268e-01,   7.95066178e-01,   1.73515204e-04],
        [  6.01844490e-03,   3.60940211e-02,   9.57887530e-01],
        [  2.91105340e-09,   8.43872083e-09,   1.00000000e+00],
        ..., 
        [  6.07628226e-01,   3.92284274e-01,   8.75087208e-05],
        [  6.60106540e-01,   3.39792520e-01,   1.00944715e-04],
        [  7.18164563e-01,   2.81617492e-01,   2.17974113e-04]],

       [[  6.91714764e-01,   3.08101416e-01,   1.83787342e-04],
        [  4.61930031e-05,   1.86851921e-05,   9.99935150e-01],
        [  3.15031091e-11,   9.73474166e-12,   1.00000000e+00],
        ..., 
        [  1.65409104e-12,   1.15175458e-12,   1.00000000e+00],
        [  4.05660488e-12,   2.58825188e-12,   1.00000000e+00],
        [  1.10248286e-11,   6.24962235e-12,   1.00000000e+00]],

       [[  7.11981487e-03,   9.92864251e-01,   1.58009570e-05],
        [  1.42539327e-03,   7.52225280e-01,   2.46349290e-01],
        [  3.80697296e-09,   7.57802638e-07,   9.99999285e-01],
        

In [82]:
inputs_test["input_length"]-2

array([61, 61, 61, 61, 61, 61, 61, 61, 61, 61, 61, 61, 61, 61, 61, 61, 61,
       61, 61, 61, 61, 61, 61, 61, 61, 61, 61, 61, 61, 61, 61, 61, 61, 61,
       61, 61, 61, 61, 61, 61, 61, 61, 61, 61, 61, 61, 61, 61, 61, 61, 61,
       61, 61, 61, 61, 61, 61, 61, 61, 61, 61, 61, 61, 61, 61, 61, 61, 61,
       61, 61, 61, 61, 61, 61, 61, 61, 61, 61, 61, 61, 61, 61, 61, 61, 61,
       61, 61, 61, 61, 61, 61, 61, 61, 61, 61, 61])

In [83]:
decode_function[0][0]

<tf.Tensor 'SparseToDense_40:0' shape=(?, ?) dtype=int64>

In [84]:
labellings1 = decode_function[0][0].eval(session=sess)

In [85]:
labellings1.shape

(96, 7)

In [86]:
preds

array([[[  4.04431164e-01,   5.70518672e-01,   2.50502191e-02],
        [  3.03568155e-01,   6.94825888e-01,   1.60586508e-03],
        [  2.04760268e-01,   7.95066178e-01,   1.73515204e-04],
        ..., 
        [  6.07628226e-01,   3.92284274e-01,   8.75087208e-05],
        [  6.60106540e-01,   3.39792520e-01,   1.00944715e-04],
        [  7.18164563e-01,   2.81617492e-01,   2.17974113e-04]],

       [[  5.42208076e-01,   4.33393270e-01,   2.43987236e-02],
        [  6.19169474e-01,   3.79417628e-01,   1.41289528e-03],
        [  6.91714764e-01,   3.08101416e-01,   1.83787342e-04],
        ..., 
        [  1.65409104e-12,   1.15175458e-12,   1.00000000e+00],
        [  4.05660488e-12,   2.58825188e-12,   1.00000000e+00],
        [  1.10248286e-11,   6.24962235e-12,   1.00000000e+00]],

       [[  1.98628649e-01,   7.91262031e-01,   1.01093641e-02],
        [  3.59845310e-02,   9.63857591e-01,   1.57893097e-04],
        [  7.11981487e-03,   9.92864251e-01,   1.58009570e-05],
        

In [87]:
preds.shape

(96, 63, 3)

In [88]:
preds[:,2:,:].shape

(96, 61, 3)

In [89]:
preds[:,2:,:]

array([[[  2.04760268e-01,   7.95066178e-01,   1.73515204e-04],
        [  6.01844490e-03,   3.60940211e-02,   9.57887530e-01],
        [  2.91105340e-09,   8.43872083e-09,   1.00000000e+00],
        ..., 
        [  6.07628226e-01,   3.92284274e-01,   8.75087208e-05],
        [  6.60106540e-01,   3.39792520e-01,   1.00944715e-04],
        [  7.18164563e-01,   2.81617492e-01,   2.17974113e-04]],

       [[  6.91714764e-01,   3.08101416e-01,   1.83787342e-04],
        [  4.61930031e-05,   1.86851921e-05,   9.99935150e-01],
        [  3.15031091e-11,   9.73474166e-12,   1.00000000e+00],
        ..., 
        [  1.65409104e-12,   1.15175458e-12,   1.00000000e+00],
        [  4.05660488e-12,   2.58825188e-12,   1.00000000e+00],
        [  1.10248286e-11,   6.24962235e-12,   1.00000000e+00]],

       [[  7.11981487e-03,   9.92864251e-01,   1.58009570e-05],
        [  1.42539327e-03,   7.52225280e-01,   2.46349290e-01],
        [  3.80697296e-09,   7.57802638e-07,   9.99999285e-01],
        

In [90]:
inputs_train["input_length"].shape

(10,)

In [91]:
inputs_test["input_length"].shape

(96,)